<a href="https://colab.research.google.com/github/b05902062/user-intention/blob/main/GenerateDataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    A token is already saved on your machine. Run `huggingface-cli whoami` to get more information or `huggingface-cli logout` if you want to log out.
    Setting a new token will erase the existing one.
    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineG

In [4]:
!pip install --upgrade datasets huggingface_hub fsspec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 12.0 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
  Attempting uninstall: datasets
    Found existing installation: datasets 2.14.4
    Uninstalling datasets-2.14.4:
      Successfully uninstalled datasets-2.14.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2025.3.2 requires fsspec==2025.3.2, but you have fsspec 2025.3.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; pla

In [26]:
new_repo_name = input("Please enter the new Hugging Face repository name (e.g., 'your-username/your-repo'): ").strip()

Please enter the new Hugging Face repository name (e.g., 'your-username/your-repo'): ZihminWang/user-intention


In [27]:
import re
import json
from datasets import load_dataset, Dataset

def transform_example(example):
    """
    Transforms a single example from the original dataset structure
    to the new desired structure. Returns None if an error occurs during transformation.
    """
    new_example = {}

    # 1. Transform 'system' field: Remove "SYSTEM: " prefix
    system_text = example['system']
    if system_text.startswith('SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n'):
        new_example['system'] = system_text[len('SYSTEM: You are a helpful assistant with access to the following functions. Use them if required -\n'):].strip()
    else:
        new_example['system'] = system_text.strip()

    # 2. Extract the first USER and first ASSISTANT block from 'chat' field
    chat_content = example['chat']

    # Regex to find the first USER block and the subsequent ASSISTANT block.
    # It captures the user's message and the assistant's immediate response.
    # We use non-greedy matching (.*?) and look for various termination markers
    # for the assistant's response: <|endoftext|>, FUNCTION RESPONSE:, or the next USER:
    chat_match = re.match(
        r'USER: (.*?)\n*\s*ASSISTANT: (.*?)(?:<\|endoftext\|>|\n*\s*FUNCTION RESPONSE:|\n*\s*USER:|$)',
        chat_content,
        re.DOTALL
    )

    if chat_match:
        user_message = chat_match.group(1).strip()
        assistant_raw = chat_match.group(2).strip()
    else:
        # Fallback for cases that might not perfectly match the regex,
        # though the provided example fits. This handles if a chat only has USER or ASSISTANT.
        print(f"Warning: Chat only has USER or ASSISTANT. Skipping example. Raw: {chat_content}")
        return None # Skip this example

    new_example['user'] = user_message

    # 3. Transform 'assistant' field based on its content
    function_call_pattern = re.compile(r'<functioncall> ({.*})')
    func_call_match = function_call_pattern.search(assistant_raw)

    if func_call_match:
        # If it's a function call, parse the JSON to get the function name
        func_call_str = func_call_match.group(1)
        try:
          # Instead of json.loads(func_call_str) which fails due to single quotes
          # within the arguments string, we'll use regex to directly extract the function name.
          name_match = re.search(r'"name": "(.*?)"', func_call_str)
          if name_match:
              function_name = name_match.group(1)
              new_example['assistant'] = json.dumps({
                "function": {
                    function_name: {
                        "intent": "true"
                    }
                }
              })
          else:
              # If function name is missing or regex fails to find it, skip example
              print(f"Warning: Function name not found in function call string. Skipping example. Raw: {func_call_str}")
              return None
        except json.JSONDecodeError as e:
            print(f"Warning: Could not parse function call JSON for example: {e}. Skipping example. Raw: {chat_content}")
            print(f"Warning: Could not parse function call JSON for example: {e}. Skipping example. Raw: {func_call_str}")
            return None # Skip this example
    else:
        # If not a function call, keep the original text, removing <|endoftext|>
        new_example['assistant'] = assistant_raw.replace('<|endoftext|>', '').strip()

    return new_example

def main():
    # Define the original dataset and the new repository name
    original_dataset_name = 'glaiveai/glaive-function-calling-v2'
    # IMPORTANT: Replace 'your-username' with your Hugging Face username
    # and 'your-new-repo-name' with the desired name for your new dataset.
    # You will need to be logged in to Hugging Face CLI or provide a token.

    print(f"Loading dataset: {original_dataset_name}")
    # Load the training split of the dataset
    dataset = load_dataset(original_dataset_name, split='train')
    print(f"Dataset loaded with {len(dataset)} examples.")

    print("Transforming dataset examples...")
    transformed_data = []
    skipped_count = 0
    for i, example in enumerate(dataset):
        try:
            transformed_example = transform_example(example)
            if transformed_example is not None:
                transformed_data.append(transformed_example)
            else:
                skipped_count += 1
        except Exception as e:
            print(f"Error transforming example {i}: {e}. Skipping row.")
            skipped_count += 1

    print(f"Finished transforming {len(transformed_data)} examples. Skipped {skipped_count} rows due to errors.")

    # Create a new Dataset object from the transformed list
    new_dataset = Dataset.from_list(transformed_data)
    print("New dataset created. Sample of transformed data:")
    # Print a sample of the transformed data for verification
    for i in range(min(3, len(new_dataset))):
        print(f"\n--- Transformed Example {i+1} ---")
        print(json.dumps(new_dataset[i], indent=2))

    print(f"\nAttempting to push transformed dataset to Hugging Face Hub: {new_repo_name}")
    # To push to the Hugging Face Hub, you need to be logged in.
    # You can do this by running `huggingface-cli login` in your terminal,
    # or by providing your token programmatically:
    # from huggingface_hub import login
    # login(token="hf_YOUR_TOKEN_HERE")
    try:
        new_dataset.push_to_hub(new_repo_name)
        print(f"Dataset successfully pushed to https://huggingface.co/datasets/{new_repo_name}")
    except Exception as e:
        print(f"Failed to push dataset to Hugging Face Hub. Please ensure you are logged in via 'huggingface-cli login' and have write permissions for '{new_repo_name}'. Error: {e}")

if __name__ == "__main__":
    main()

Loading dataset: glaiveai/glaive-function-calling-v2
Dataset loaded with 112960 examples.
Transforming dataset examples...
Finished transforming 112960 examples. Skipped 0 rows due to errors.
New dataset created. Sample of transformed data:

--- Transformed Example 1 ---
{
  "system": "{\n    \"name\": \"get_exchange_rate\",\n    \"description\": \"Get the exchange rate between two currencies\",\n    \"parameters\": {\n        \"type\": \"object\",\n        \"properties\": {\n            \"base_currency\": {\n                \"type\": \"string\",\n                \"description\": \"The currency to convert from\"\n            },\n            \"target_currency\": {\n                \"type\": \"string\",\n                \"description\": \"The currency to convert to\"\n            }\n        },\n        \"required\": [\n            \"base_currency\",\n            \"target_currency\"\n        ]\n    }\n}",
  "user": "Can you book a flight for me from New York to London?",
  "assistant": "I

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/113 [00:00<?, ?ba/s]

Uploading...:   0%|          | 0.00/43.9M [00:00<?, ?B/s]

Dataset successfully pushed to https://huggingface.co/datasets/ZihminWang/user-intention
